Introductie

P1

P2

P3

P4

## De invloed van klimaat en geografie op voedselproductie

Niet elk land beschikt over dezelfde natuurlijke omstandigheden om voedsel te produceren. Factoren zoals neerslag, temperatuur en klimaatzones bepalen welke gewassen efficiënt kunnen groeien. In landen met een ongunstig klimaat moeten gewassen onder intensievere en vaak minder duurzame omstandigheden worden verbouwd.
Daarnaast speelt de geografische ligging een rol in hoe voedsel wordt verhandeld: landen die ver van exportmarkten liggen, hebben hogere transportkosten en extra uitstoot, wat hun voedselsysteem minder duurzaam maakt.
In dit perspectief onderzoeken we hoe deze geografische factoren — natuurlijke omstandigheden en ligging ten opzichte van wereldmarkten — bijdragen aan verschillen in de duurzaamheid van nationale voedselsystemen.

### Klimaatzones en natuurlijke omstandigheden beïnvloeden duurzaamheid
Landen verschillen sterk in klimaat. Tropische regio’s kennen bijvoorbeeld hoge temperaturen en regelmatige regenval, terwijl droge of koude gebieden kampen met beperkte groeiseizoenen en lagere landbouwopbrengsten per hectare.
In warmere, vruchtbare klimaatzones kunnen gewassen natuurlijk groeien, met weinig extra input. In koudere of drogere gebieden zijn irrigatie, verwarming, kunstmest of import van hulpbronnen nodig om dezelfde opbrengst te halen. Deze extra inputs verhogen de uitstoot per kilogram geproduceerd voedsel.
Daarom is het voedselsysteem in sommige landen per definitie minder duurzaam, simpelweg door hun ligging in een ongunstige klimaatzone.

In [56]:
import pandas as pd
import plotly.express as px

# 📁 Laad de datasets
neerslag_df = pd.read_csv("datasets/API_AG.LND.PRCP.MM_DS2_en_csv_v2_4487.csv", skiprows=4)
productie_df = pd.read_csv("datasets/Production_Crops_Livestock_filtered.csv")
emissies_crops_df = pd.read_csv("datasets/Emissions_crops_filtered.csv")
emissies_livestock_df = pd.read_csv("datasets/Emissions_livestock_filtered.csv")

# 🌧️ Verwerk neerslagdata
neerslag_long = neerslag_df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    var_name="Year",
    value_name="Precipitation_mm"
)
neerslag_long = neerslag_long[["Country Name", "Year", "Precipitation_mm"]]
neerslag_long["Year"] = pd.to_numeric(neerslag_long["Year"], errors="coerce")
neerslag_long = neerslag_long.dropna(subset=["Year", "Precipitation_mm"])
neerslag_long["Country Name"] = neerslag_long["Country Name"].replace({
    "United States": "United States of America"
})

# 🚜 Verwerk productiegegevens
productie_filtered = productie_df[productie_df["Element"] == "Production"]
productie_aggregated = productie_filtered.groupby(["Area", "Year"])["Value"].sum().reset_index()
productie_aggregated.rename(columns={"Area": "Country Name", "Value": "Production_tonnes"}, inplace=True)
productie_aggregated["Year"] = pd.to_numeric(productie_aggregated["Year"], errors="coerce")
productie_aggregated["Country Name"] = productie_aggregated["Country Name"].replace({
    "Netherlands (Kingdom of the)": "Netherlands"
})

# 🌱🐄 Combineer gewas- en veeteeltuitstoot
emissies_crops_total = emissies_crops_df.groupby(["Area", "Year"])["Value"].sum().reset_index()
emissies_livestock_total = emissies_livestock_df.groupby(["Area", "Year"])["Value"].sum().reset_index()

emissies_total = pd.merge(
    emissies_crops_total, emissies_livestock_total,
    on=["Area", "Year"], how="outer", suffixes=("_crops", "_livestock")
)
emissies_total["Emissions_total"] = (
    emissies_total["Value_crops"].fillna(0) + emissies_total["Value_livestock"].fillna(0)
)
emissies_total = emissies_total[["Area", "Year", "Emissions_total"]]
emissies_total.rename(columns={"Area": "Country Name"}, inplace=True)
emissies_total["Year"] = pd.to_numeric(emissies_total["Year"], errors="coerce")

# 🔁 Merge alles samen op land + jaar
emissie_productie_df = pd.merge(emissies_total, productie_aggregated, on=["Country Name", "Year"])
emissie_productie_df = pd.merge(emissie_productie_df, neerslag_long, on=["Country Name", "Year"])

# 🔎 Filter op de 7 landen
landen_selectie = [
    "Nigeria", "South Africa", "India", "Japan",
    "United States of America", "Brazil", "Indonesia"
]
emissie_productie_df = emissie_productie_df[emissie_productie_df["Country Name"].isin(landen_selectie)]

# ➗ Bereken uitstoot per ton
emissie_productie_df["Emissions_per_ton"] = (
    emissie_productie_df["Emissions_total"] / emissie_productie_df["Production_tonnes"]
)

# 📈 Maak scatterplot
fig = px.scatter(
    emissie_productie_df,
    x="Precipitation_mm",
    y="Emissions_per_ton",
    animation_frame="Year",
    color="Country Name",
    text="Country Name",
    hover_name="Country Name",
    title="Uitstoot per ton geproduceerd vs Neerslag (per jaar)",
    labels={
        "Precipitation_mm": "Neerslag (mm/jaar)",
        "Emissions_per_ton": "Uitstoot per ton geproduceerd (Mt/ton)"
    }
)

fig.update_traces(
    textposition='top center',
    marker=dict(size=14)
)

fig.update_layout(
    xaxis=dict(range=[0, 3000]),
    yaxis=dict(range=[0, 6e-7])
)

fig.show()



/var/folders/82/txt4_nkx0l76kq68c7g_klqc0000gn/T/ipykernel_32679/3348794567.py:6: DtypeWarning:

Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.



#### Analyse: Uitstoot per ton geproduceerd vs Neerslag

Deze visualisatie toont hoe efficiënt landen voedsel produceren in termen van uitstoot. Op de x-as staat de jaarlijkse neerslag (in mm), en op de y-as de uitstoot uit landbouw omgerekend naar **uitstoot per geproduceerde ton voedsel** (Mt CO₂e/ton). Elk punt vertegenwoordigt één land in een bepaald jaar. We volgen Brazilië, India, Indonesië, Japan, Nigeria, Zuid-Afrika en 
de Verenigde Staten.

##### Waarom deze landen?

In deze analyse beperken we ons tot zeven landen: Brazilië, India, Indonesië, Japan, Nigeria, Zuid-Afrika en de Verenigde Staten. Deze selectie is bewust gemaakt om drie redenen:

1. **Contrasterende landbouwsystemen**  
   Elk van deze landen vertegenwoordigt een fundamenteel ander landbouwmodel: van grootschalige industriële productie (VS), tot extensieve plantaardige landbouw (India), tot tropisch kleinschalig boerenbedrijf (Nigeria, Indonesië). Door juist deze contrasten te vergelijken, ontstaat inzicht in hoe verschillende strategieën leiden tot uiteenlopende emissiepatronen, los van natuurlijke omstandigheden.

2. **Relevantie voor mondiale voedselvoorziening én uitstoot**  
   De gekozen landen behoren tot de grootste producenten van voedsel in de wereld (India, VS, Brazilië), of zijn door hun bevolkingsomvang of economische groei belangrijk voor de toekomst van landbouwtransitie (zoals Indonesië, Nigeria). Ze dragen significant bij aan zowel de wereldwijde voedselproductie als de landbouwgerelateerde uitstoot, wat hen tot relevante cases maakt.

3. **Beheersbare maar representatieve vergelijking**  
   Door te focussen op zeven landen kunnen we trends in detail analyseren, zonder dat de vergelijking verwatert. Een analyse van alle landen zou leiden tot visuele ruis en afleiding van het doel: het blootleggen van fundamentele verschillen in landbouw-efficiëntie. De gekozen landen bestrijken vijf continenten, meerdere klimaatzones, en uiteenlopende sociaaleconomische structuren, waardoor ze samen een krachtig representatief beeld geven van mondiale landbouwdynamiek.

Kortom: deze landen zijn niet willekeurig gekozen, maar geselecteerd om de kernvraag van deze analyse – **hoe efficiënt produceren landen voedsel in termen van uitstoot?** – zo scherp en inzichtelijk mogelijk te beantwoorden.


##### Wat valt op?

- **De Verenigde Staten heeft structureel een hoge uitstoot per ton geproduceerd.**  
  Hoewel de VS een grote landbouwsector heeft, is de uitstoot per ton voedsel aanzienlijk hoger dan in bijvoorbeeld India of Brazilië. Dit wijst waarschijnlijk op een uitstootintensieve landbouwpraktijk, zoals intensieve veeteelt (runderen, varkens), gebruik van grote hoeveelheden kunstmest, en een hoge mate van mechanisatie en energieverbruik.

- **India en Brazilië combineren hoge productie met relatief lage uitstoot.**  
  India produceert in veel jaren zelfs meer dan de VS, maar doet dit met een aanzienlijk lagere uitstoot per ton. Dit suggereert een meer plantaardige en extensieve landbouw, of efficiënter gebruik van middelen. Ook Brazilië scoort opvallend goed qua uitstoot-efficiëntie, ondanks hoge neerslag.

- **Japan en Zuid-Afrika zitten qua efficiëntie in de middenmoot.**  
  Japan is technologisch geavanceerd maar kleinschalig, wat leidt tot stabiele maar gematigde uitstoot per ton. Zuid-Afrika toont sterke fluctuaties, wat kan wijzen op klimaatinvloeden (zoals droogte), economische instabiliteit of structurele verschillen in landbouwtypes per jaar.

- **Indonesië en Nigeria produceren weinig, en stoten relatief weinig uit.**  
  In beide landen is de landbouw kleinschaliger en minder geïndustrialiseerd. Dat leidt tot lage totale productie, maar ook tot lage uitstoot per ton. Wellicht is er beperkte toegang tot landbouwtechnologie of infrastructuur.

##### Waarom stoten sommige landen meer uit per ton dan andere?

Een hoge uitstoot per ton geproduceerd voedsel kan verschillende oorzaken hebben:
- Dominantie van veeteelt (vooral rundvlees en zuivel -> hoge methaanuitstoot)
- Inefficiënt gebruik van landbouwgrond of input (bijv. overmatig gebruik van kunstmest)
- Mechanisatie die veel fossiele energie verbruikt
- Verspilling in de keten of lage opbrengst per hectare

##### Conclusie

De visualisatie laat overtuigend zien dat **meer neerslag niet automatisch leidt tot efficiëntere of duurzamere landbouw.** India toont aan dat een droog(achtig) klimaat niet hoeft te betekenen dat productie inefficiënt is. Tegelijkertijd laat de VS zien dat landbouwkeuzes een grotere impact hebben dan klimaat: ondanks goede omstandigheden is de uitstoot per ton daar hoog.

Efficiëntie in landbouw wordt dus grotendeels bepaald door **productiestrategie**, niet door geografie. Wie wat produceert – en hóé – bepaalt uiteindelijk de ecologische voetafdruk.



### Internationale handelsstromen beïnvloeden nationale uitstoot

De uitstoot van een nationaal voedselsysteem zegt niet alleen iets over wat er binnen dat land geconsumeerd wordt, maar ook over wat er geproduceerd wordt voor de wereldmarkt. Sommige landen produceren voedsel voornamelijk voor export. In die gevallen wordt de uitstoot lokaal geregistreerd, terwijl de consumptie elders plaatsvindt. 

Omdat deze landen produceren voor internationale markten, ligt hun landbouwuitstoot vaak hoger dan landen met vergelijkbare bevolkingsaantallen of eetgewoonten. Dit betekent dat de uitstoot van een land niet per se iets zegt over het lokale dieet of de binnenlandse voedselvoorziening. Het weerspiegelt eerder hun positie in het mondiale voedselsysteem. 

Dus landbouwuitstoot weerspiegelt niet enkel lokale consumptie, maar wordt sterk beïnvloed door de internationale vraag – landen met hoge export produceren meer en stoten daardoor meer uit, vaak voor andermans bord.


In [ ]:
import pandas as pd
import plotly.graph_objects as go


emissions_animal = pd.read_csv("datasets/FAOSTAT_data_emissions_animal_6-23-2025.csv")
emissions_crops = pd.read_csv("datasets/FAOSTAT_data_emissions_crops_6-23-2025-2.csv")
export_matrix = pd.read_csv("datasets/matrix_export.csv")
population = pd.read_csv("datasets/Population_E_All_Data_(Normalized).csv")


population_clean = population[
    (population["Element"] == "Total Population - Both sexes") &
    (population["Area"].isin(["Argentina", "Brazil", "India", "Thailand", "Netherlands", "United States of America"]))
][["Area", "Year", "Value"]].rename(columns={"Value": "Population (1000s)"})
population_clean["Year"] = population_clean["Year"].astype(int)


animal = emissions_animal[
    emissions_animal["Area"].isin(["Argentina", "Brazil", "India", "Thailand", "Netherlands", "United States of America"]) &
    emissions_animal["Element"].str.contains("Emissions")
]
crops = emissions_crops[
    emissions_crops["Area"].isin(["Argentina", "Brazil", "India", "Thailand", "Netherlands", "United States of America"]) &
    emissions_crops["Element"].str.contains("Emissions")
]

emissions_all = pd.concat([animal, crops])
emissions_all["Year"] = emissions_all["Year"].astype(int)
emissions_all = emissions_all.groupby(["Area", "Year"])["Value"].sum().reset_index()
emissions_all.rename(columns={"Value": "Total Emissions (kt CO2eq)"}, inplace=True)


export_volume = export_matrix[
    (export_matrix["Element"] == "Export quantity") &
    (export_matrix["Reporter Countries"].isin(["Argentina", "Brazil", "India", "Thailand", "Netherlands", "United States of America"]))
][["Reporter Countries", "Year", "Value"]]
export_volume = export_volume.groupby(["Reporter Countries", "Year"]).sum().reset_index()
export_volume.rename(columns={
    "Reporter Countries": "Area",
    "Value": "Total Export Volume (tonnes)"
}, inplace=True)
export_volume["Year"] = export_volume["Year"].astype(int)


df = emissions_all.merge(population_clean, on=["Area", "Year"], how="left")
df = df.merge(export_volume, on=["Area", "Year"], how="left")


df = df[df["Year"] >= 1985]


df["Emissions per Capita (tonnes CO2eq)"] = (df["Total Emissions (kt CO2eq)"] * 1000) / (df["Population (1000s)"] * 1000)
df["Export per Capita (tonnes)"] = df["Total Export Volume (tonnes)"] / (df["Population (1000s)"] * 1000)
df["Emissions per Export Tonne (kg CO2eq/tonne)"] = (df["Total Emissions (kt CO2eq)"] * 1_000_000) / df["Total Export Volume (tonnes)"]


metrics = {
    "Total Emissions (kt CO2eq)": "Total Emissions (kt CO2eq)",
    "Emissions per Capita (tonnes CO2eq)": "Emissions per Capita (tonnes CO2eq)",
    "Total Export Volume (tonnes)": "Total Export Volume (tonnes)",
    "Export per Capita (tonnes)": "Export per Capita (tonnes)",
    "Emissions per Export Tonne (kg CO2eq/tonne)": "Emissions per Export Tonne (kg CO2eq/tonne)"
}


fig = go.Figure()
for metric_index, (metric_name, col) in enumerate(metrics.items()):
    for country in df["Area"].unique():
        subset = df[df["Area"] == country]
        visible = (metric_index == 0)
        fig.add_trace(go.Scatter(
            x=subset["Year"],
            y=subset[col],
            mode='lines',
            name=country,
            visible=visible,
            legendgroup=country,
            showlegend=True
        ))


n_countries = len(df["Area"].unique())
dropdown_buttons = []
for i, (metric_name, _) in enumerate(metrics.items()):
    visibility = [False] * len(metrics) * n_countries
    for j in range(n_countries):
        visibility[i * n_countries + j] = True
    dropdown_buttons.append(dict(
        label=metric_name,
        method="update",
        args=[{"visible": visibility},
              {"yaxis": {"title": metric_name}}]
    ))

fig.update_layout(
    updatemenus=[{
        "buttons": dropdown_buttons,
        "direction": "down",
        "showactive": True,
        "x": 1.03,
        "xanchor": "left",
        "y": 0.95,
        "yanchor": "top",
        "pad": {"r": 0, "t": 0},
        "font": {"size": 11},
        "bgcolor": "white"
    }],
    title="Agricultural Emissions & Exports (1985–2020)",
    xaxis_title="Year",
    yaxis_title="Total Emissions (kt CO2eq)",
    legend_title="Country",
    legend=dict(
        x=1.03,
        xanchor="left",
        y=0.8,
        yanchor="top"
    ),
    margin=dict(r=160, l=80, t=80, b=60),
    height=600,
    width=900
)




fig.show()




De geografische ligging van een land (en de bijbehorende klimaatzones) beïnvloedt welke gewassen en dieren men produceert — en dat heeft directe impact op de landbouwgerelateerde CO₂-uitstoot per capita.

In [ ]:
import pandas as pd
import plotly.express as px

emissions_crops = pd.read_csv("datasets/FAOSTAT_emissions_cropss.csv")
emissions_livestock = pd.read_csv("datasets/FAOSTAT_emissions_livestock.csv")
population = pd.read_csv("datasets/Population_E_All_Data_(Normalized).csv")

landen = [
    "Brazil", "Indonesia", "Kenya", "India",
    "Netherlands (Kingdom of the)", "Nigeria", "South Africa", "United States of America"
]

# Emissies per gewas/diergroep optellen
crops_filtered = emissions_crops[emissions_crops["Area"].isin(landen)]
livestock_filtered = emissions_livestock[emissions_livestock["Area"].isin(landen)]
emissions = pd.concat([crops_filtered, livestock_filtered])
emissions = emissions.groupby(["Area", "Year"], as_index=False)["Value"].sum()
emissions.rename(columns={"Value": "Total Emissions (kt CO2eq)"}, inplace=True)

# Bevolking
pop_filtered = population[
    (population["Area"].isin(landen)) &
    (population["Element"] == "Total Population - Both sexes")
][["Area", "Year", "Value"]].rename(columns={"Value": "Population (1000s)"})

emissions["Year"] = emissions["Year"].astype(int)
pop_filtered["Year"] = pop_filtered["Year"].astype(int)
df = emissions.merge(pop_filtered, on=["Area", "Year"], how="left")
df["Emissions per Capita (tonnes CO2eq)"] = (df["Total Emissions (kt CO2eq)"] * 1000) / (df["Population (1000s)"] * 1000)

fig = px.line(
    df[df["Year"] >= 1985],
    x="Year",
    y="Emissions per Capita (tonnes CO2eq)",
    color="Area",
    title="Landbouwgerelateerde CO₂-uitstoot per capita (1985–2020)",
    labels={"Emissions per Capita (tonnes CO2eq)": "Uitstoot per Capita (tonnes CO₂eq)", "Area": "Land"}
)

fig.update_layout(
    legend_title="Land",
    height=600
)

fig.show()


In [41]:
import pandas as pd
import plotly.graph_objects as go

# A) Neerslagdata laden
rain = pd.read_csv("datasets/API_AG.LND.PRCP.MM_DS2_en_csv_v2_4487.csv", skiprows=4)
rain = rain.melt(id_vars=["Country Name"], var_name="Year", value_name="Precip_mm")
rain = rain[rain["Year"].str.fullmatch(r"\d{4}")]
rain["Year"] = rain["Year"].astype(int)
rain = rain[rain["Year"] >= 1985]
rain_avg = rain.groupby("Country Name")["Precip_mm"].mean().reset_index()

# B) Klimaatzones laden (csv uit ChatGPT)
climate = pd.read_csv("datasets/koeppen_geiger_climate_zones.csv")  # <-- verander dit pad indien nodig
# Verwacht: kolommen 'Country Name', 'ClimateCode'

print(climate.columns)


# C) Kleurmapping (Köppen–Geiger kleurenlegenda)
climate_map = {
    'Af': '#0000FF', 'Am': '#1E90FF', 'Aw': '#00BFFF', 'As': '#87CEFA',
    'BWh': '#FF0000', 'BWk': '#FFB6C1', 'BSh': '#FFA500', 'BSk': '#FFFF00',
    'Csa': '#FFFF99', 'Csb': '#FFD700', 'Csc': '#DAA520',
    'Cwa': '#ADFF2F', 'Cwb': '#90EE90', 'Cwc': '#006400',
    'Cfa': '#7CFC00', 'Cfb': '#32CD32', 'Cfc': '#228B22',
    'Dsa': '#FF69B4', 'Dsb': '#FF1493', 'Dsc': '#C71585', 'Dsd': '#8B008B',
    'Dwa': '#9370DB', 'Dwb': '#7B68EE', 'Dwc': '#6A5ACD', 'Dwd': '#483D8B',
    'Dfa': '#00FFFF', 'Dfb': '#40E0D0', 'Dfc': '#00CED1', 'Dfd': '#008B8B',
    'ET': '#A9A9A9', 'EF': '#696969'
}

climate = climate[climate["ClimateCode"].isin(climate_map)]  # filter onbekende codes eruit
climate["Color"] = climate["ClimateCode"].map(climate_map)

# D) Samenvoegen van neerslag en klimaatdata
world = pd.merge(rain_avg, climate, on="Country Name", how="left")
world["ClimateIndex"], labels = pd.factorize(world["ClimateCode"])

# E) Plotly visualisatie
fig = go.Figure()

# 1. Neerslagkaart
fig.add_trace(go.Choropleth(
    locations=world["Country Name"],
    locationmode="country names",
    z=world["Precip_mm"],
    colorscale="Blues",
    colorbar_title="mm/jaar",
    visible=True,
    name="Neerslag"
))

# 2. Klimaatzonekaart
fig.add_trace(go.Choropleth(
    locations=world["Country Name"],
    locationmode="country names",
    z=world["ClimateIndex"],
    text=world["ClimateCode"],
    colorscale=[(i/(len(labels)-1), climate_map[code]) for i, code in enumerate(labels)],
    colorbar=dict(
        tickvals=list(range(len(labels))),
        ticktext=list(labels),
        title="Köppen–Geiger"
    ),
    visible=False,
    name="Klimaatzone"
))

# 3. Dropdown menu
fig.update_layout(
    updatemenus=[{
        "buttons": [
            {"method": "update", "label": "Neerslag", "args": [{"visible": [True, False]}, {"title": "Gemiddelde Neerslag per Jaar (1985–2020)"}]},
            {"method": "update", "label": "Klimaatzones", "args": [{"visible": [False, True]}, {"title": "Köppen–Geiger Klimaatzones per Land"}]}
        ],
        "direction": "down",
        "x": 0, "xanchor": "left", "y": 1.1, "yanchor": "top"
    }],
    geo=dict(showframe=False, showcoastlines=True, projection_type='equirectangular'),
    margin=dict(l=0, r=0, t=50, b=0),
    height=650
)

fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/koeppen_geiger_climate_zones.csv'

In [42]:
# import pandas as pd
# import plotly.express as px
# import numpy as np

# # Haal alle landen op uit de gapminder dataset (van Plotly)
# landen = px.data.gapminder()['country'].unique()

# # Kies willekeurig een klimaatzone per land
# klimaatzones = ["Af", "Am", "Aw", "BWh", "BWk", "Cfa", "Cfb", "Dfa", "Dfb", "Dfc", "ET", "EF"]
# np.random.seed(42)  # Voor herhaalbaarheid
# toewijzingen = np.random.choice(klimaatzones, size=len(landen))

# # Maak dataframe
# df = pd.DataFrame({
#     "Country Name": landen,
#     "ClimateCode": toewijzingen
# })

# # Kleuren toewijzen aan klimaatzones
# kleurmap = {
#     "Af": "#1f77b4", "Am": "#17becf", "Aw": "#ff7f0e",
#     "BWh": "#d62728", "BWk": "#e377c2",
#     "Cfa": "#2ca02c", "Cfb": "#98df8a",
#     "Dfa": "#8c564b", "Dfb": "#9467bd", "Dfc": "#7f7f7f",
#     "ET": "#bcbd22", "EF": "#aec7e8"
# }

# # Visualisatie
# fig = px.choropleth(
#     df,
#     locations="Country Name",
#     locationmode="country names",
#     color="ClimateCode",
#     title="Wereldkaart met Willekeurige Köppen–Geiger Klimaatzones",
#     color_discrete_map=kleurmap
# )

# fig.update_layout(
#     legend_title_text="Klimaatzone (Köppen–Geiger)",
#     margin={"r":0,"t":30,"l":0,"b":0}
# )

# fig.show()


In [40]:
import pandas as pd
import plotly.graph_objects as go

# -- Inladen data --
temp_df = pd.read_csv("datasets/average-monthly-surface-temperature.csv")
rain_df = pd.read_csv("datasets/average-precipitation-per-year.csv")

# -- Landen & naamcorrecties --
landen = [
    "Brazil", "Canada", "China", "India", "Indonesia", "Kenya",
    "Netherlands (Kingdom of the)", "Nigeria", "South Africa", "United States of America"
]
land_mapping = {
    "Netherlands (Kingdom of the)": "Netherlands",
    "United States of America": "United States"
}
temp_df["Entity"] = temp_df["Entity"].replace(land_mapping)
rain_df["Entity"] = rain_df["Entity"].replace(land_mapping)
landen = [land_mapping.get(land, land) for land in landen]

# -- Temperatuur verwerken --
temp_df_filtered = temp_df[temp_df["Entity"].isin(landen)].copy()
temp_df_filtered["Year"] = temp_df_filtered["year"]
temp_avg = temp_df_filtered.groupby(["Entity", "Year"])["Average surface temperature"].mean().reset_index()

# -- Neerslag verwerken --
rain_df_filtered = rain_df[rain_df["Entity"].isin(landen)].copy()
rain_df_filtered = rain_df_filtered[["Entity", "Year", "Annual precipitation"]]

# -- Samenvoegen --
merged = pd.merge(temp_avg, rain_df_filtered, on=["Entity", "Year"])

# -- Interactieve visualisatie met dropdown --
fig = go.Figure()
buttons = []

for i, land in enumerate(landen):
    subset = merged[merged["Entity"] == land]

    fig.add_trace(go.Bar(
        x=subset["Year"],
        y=subset["Annual precipitation"],
        name="Neerslag (mm)",
        marker_color="skyblue",
        yaxis="y1",
        visible=(i == 0)
    ))
    fig.add_trace(go.Scatter(
        x=subset["Year"],
        y=subset["Average surface temperature"],
        name="Temperatuur (°C)",
        mode="lines",
        line=dict(color="red"),
        yaxis="y2",
        visible=(i == 0)
    ))

    visibility = [False] * (2 * len(landen))
    visibility[2 * i] = True
    visibility[2 * i + 1] = True

    buttons.append(dict(
        label=land,
        method="update",
        args=[{"visible": visibility},
              {"title": f"Neerslag en temperatuur in {land}"}]
    ))

fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    title=f"Neerslag en temperatuur in {landen[0]}",
    xaxis=dict(title="Jaar"),
    yaxis=dict(title="Neerslag (mm)", range=[0, 3500]),
    yaxis2=dict(
    title="Temperatuur (°C)",
    overlaying="y",
    side="right",
    range=[-7, 30]
),
    legend=dict(x=0.5, xanchor="center", y=-0.2, orientation="h"),
    height=600
)

fig.show()
